In [1]:
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
news_df = pd.read_csv("../../data/raw_analyst_ratings.csv", parse_dates=["date"])
news_df = news_df[news_df['stock'].isin(['AAPL'])]
stock_df = pd.read_csv("../../data/yfinance_data/AAPL_historical_data.csv", parse_dates=["Date"])



In [3]:
news_df.columns = news_df.columns.str.lower()
stock_df.columns = stock_df.columns.str.lower()

In [4]:
print(news_df.head(2))

      unnamed: 0                                           headline  \
6680        7120  Tech Stocks And FAANGS Strong Again To Start D...   
6681        7121      10 Biggest Price Target Changes For Wednesday   

                                                    url   publisher  \
6680  https://www.benzinga.com/government/20/06/1622...  JJ Kinahan   
6681  https://www.benzinga.com/analyst-ratings/price...  Lisa Levin   

                           date stock  
6680  2020-06-10 11:33:26-04:00  AAPL  
6681  2020-06-10 08:14:08-04:00  AAPL  


In [5]:
print(stock_df.head(2))

        date      open      high       low     close  adj close     volume  \
0 1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
1 1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781  175884800   

   dividends  stock splits  
0        0.0           0.0  
1        0.0           0.0  


In [6]:
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity

news_df["sentiment"] = news_df["headline"].apply(get_sentiment)

In [7]:
print(news_df.head(8))

      unnamed: 0                                           headline  \
6680        7120  Tech Stocks And FAANGS Strong Again To Start D...   
6681        7121      10 Biggest Price Target Changes For Wednesday   
6682        7122  Benzinga Pro's Top 5 Stocks To Watch For Wed.,...   
6683        7123  Deutsche Bank Maintains Buy on Apple, Raises P...   
6684        7124  Apple To Let Users Trade In Their Mac Computer...   
6685        7125  Big Tech Reaches New Record Heights At The Sto...   
6686        7126          Why Apple's Stock Is Trading Higher Today   
6687        7127  Apple Could Announce In-House Chips For Macs A...   

                                                    url          publisher  \
6680  https://www.benzinga.com/government/20/06/1622...         JJ Kinahan   
6681  https://www.benzinga.com/analyst-ratings/price...         Lisa Levin   
6682  https://www.benzinga.com/short-sellers/20/06/1...  Benzinga Newsdesk   
6683  https://www.benzinga.com/news/20/06/162198

In [8]:
# 3. Aggregate sentiment by date
daily_sentiment = news_df.groupby("date")["sentiment"].mean().reset_index()

In [9]:
# 4. Compute daily returns from stock data
stock_df.sort_values("date", inplace=True)
stock_df["daily_return"] = stock_df["close"].pct_change()

In [10]:
print(stock_df.head(2))

        date      open      high       low     close  adj close     volume  \
0 1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
1 1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781  175884800   

   dividends  stock splits  daily_return  
0        0.0           0.0           NaN  
1        0.0           0.0     -0.052171  


In [11]:
# 5. Merge sentiment and return data
merged_df = pd.merge(daily_sentiment, stock_df[["date", "daily_return"]], on="date", how="inner")

ValueError: You are trying to merge on object and datetime64[ns] columns for key 'date'. If you wish to proceed you should use pd.concat